In [ ]:
import pandas as pd
import numpy as np
import os

## Creating stock market holiday calendar

In [ ]:
from pandas.tseries.holiday import nearest_workday, AbstractHolidayCalendar, Holiday, USMartinLutherKingJr, USPresidentsDay, GoodFriday, USMemorialDay, USLaborDay, USThanksgivingDay

In [ ]:
class USTradingHolidaysCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday(
            'NewYearsDay',
            month=1,
            day=1,
            observance=nearest_workday
        ),
        USMartinLutherKingJr,
        USPresidentsDay,
        GoodFriday,
        USMemorialDay,
        Holiday(
            'Juneteenth National Independence Day',
            month=6,
            day=19,
            observance=nearest_workday,
        ),
        Holiday(
            'USIndependenceDay',
            month=7,
            day=4,
            observance=nearest_workday
        ),
        USLaborDay,
        USThanksgivingDay,
        Holiday(
            'Christmas',
            month=12,
            day=25,
            observance=nearest_workday
        ),
        Holiday(
            'Hurricane Gloria',
            month=9,
            day=27,
            year=1985,
            observance=nearest_workday
        ),
        Holiday(
            'Memory of President Richard M.Nixon',
            month=4,
            day=27,
            year=1994,
            observance=nearest_workday
        ),
        Holiday(
            'Close due to 9-11',
            month=9,
            day=11,
            year=2001,
            observance=nearest_workday
        ),
        Holiday(
            'Close due to 9-11',
            month=9,
            day=12,
            year=2001,
            observance=nearest_workday
        ),
        Holiday(
            'Close due to 9-11',
            month=9,
            day=13,
            year=2001,
            observance=nearest_workday
        ),
        Holiday(
            'Close due to 9-11',
            month=9,
            day=14,
            year=2001,
            observance=nearest_workday
        ),
        Holiday(
            'Reagan funeral',
            month=6,
            day=11,
            year=2004,
            observance=nearest_workday
        ),
        Holiday(
            'President Ford',
            month=1,
            day=2,
            year=2007,
            observance=nearest_workday
        ),
        Holiday(
            'Hurriance Sandy',
            month=10,
            day=29,
            year=2012,
            observance=nearest_workday
        ),
        Holiday(
            'Hurriance Sandy2',
            month=10,
            day=30,
            year=2012,
            observance=nearest_workday
        )
        
    ]

In [ ]:
cal = USTradingHolidaysCalendar()
holidays = cal.holidays(start='1980-01-01', end='2017-12-31')
holidays = [str(holidays[i].date()) for i in range(len(holidays))]

## For Any Company

In [ ]:
# assign directory
directory = 'Data\ToClean'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        #Skip empty File
        if os.stat(f).st_size == 0:
            continue
        
        #Finding missing dates
        df = pd.read_csv(f, sep = ",")

  
        start_date = df["Date"].iloc[0]
        end_date =  df["Date"].iloc[-1]
        df["Date"] = pd.to_datetime(df["Date"])
        
        #Keep dates after 1980s
        df = df.loc[df["Date"] >= pd.to_datetime("1980-1-1")]
        df = df.reset_index(drop=True)
        
        df = df.set_index("Date")
        df_date = df.index
        df_date_range = pd.to_datetime(pd.date_range(start = start_date, end = end_date))
        df_missing_dates = df_date_range.difference(df_date)

        df_missing_dates = df_missing_dates.to_series()
        df_missing_dates = df_missing_dates.dt.day_of_week
        df_missing_dates = pd.DataFrame(df_missing_dates)
        
        df_non_weekends = []
        for i in range(len(df_missing_dates)):
            if df_missing_dates[0][i] != 5 and df_missing_dates[0][i] != 6:
                df_non_weekends.append(str(df_missing_dates.iloc[i].name.date()))
        
        df_missing_non_hols = []
        for i in range(len(df_non_weekends)):
            if df_non_weekends[i] not in holidays:
                df_missing_non_hols.append(df_non_weekends[i])

        for date in df_missing_non_hols:
            pd.concat([df, pd.DataFrame(index=[pd.to_datetime(date)])])
        
        df = df.sort_index()
        df = df.ffill()

        #Add Variable
        df_v1 = df.copy(deep=True)
        
        #adding year and month column
        df_v1["year"] = df_v1.index.year
        df_v1["month"] = df_v1.index.month
        df_v1["day"] = df_v1.index.day_of_week

        '''
        #Add first diff col
        df_v1["first_diff"] = df_v1["Close"].diff()
        #Add daily average col
        df_v1["daily_avg"] = (df_v1["Open"] + df_v1["High"] + df_v1["Low"] + df_v1["Close"])/4
        '''
        #high - close
        df_v1["h-c"] = df_v1["High"] - df_v1["Close"]

        #close - low
        df_v1["c-l"] = df_v1["Close"] - df_v1["Low"]

        #close - open
        df_v1["c-o"] = df_v1["Close"] - df_v1["Open"]

        #getting exponential moving averages
        for i in [10, 50, 100, 200]:
          df_v1["ewm" + str(i)] = df_v1["Close"].ewm(span = i, adjust = True).mean()
          

        #getting moving averages
        for i in [10, 50, 100, 200]:
          df_v1["ma" + str(i)] = df_v1["Close"].rolling(i).mean()

        #getting moving standard devs
        for i in [10, 50, 100, 200]:
          df_v1["std" + str(i)] = df_v1["Close"].rolling(i).std()

        #bollinger band bounds
        std20 = df_v1['Close'].rolling(20).std()
        mean20 = df_v1["Close"].rolling(20).mean()
        df_v1["UB"] = mean20 + 2*std20
        df_v1["LB"] = mean20 - 2*std20

        #getting lags
        for i in range(2,51):
          data["lag" + str(i)] = data["Close"].shift(i-1)
        
        #dropping open, high, low to avoid multicollinearity
        df_v1 = df_v1.drop('Open', axis=1)
        df_v1 = df_v1.drop('High', axis=1)
        df_v1 = df_v1.drop('Low', axis=1)

        #Create a target for close price (tmr close) by shifting it down
        df_v1["Tomorrow_Close"] = df_v1["Close"].shift(-1)

        #sync date variable columns with target
        df_v1 = df_v1.reset_index()
        df["Date"] = df["Date"].shift(-1)
        df["year"] = df["year"].shift(-1)
        df["month"]= df["month"].shift(-1)
        df["day"] = df["day"].shift(-1)



        '''
        #Create diff(of closed price) from rolling average
        df_v1["diff_CRA_7d"] = df_v1["Close"] - df_v1["Close_7d_ra"]
        df_v1["diff_CRA_30d"] = df_v1["Close"] - df_v1["Close_30d_ra"]
        df_v1["diff_CRA_90d"] = df_v1["Close"] - df_v1["Close_90d_ra"]
        df_v1["diff_CRA_365d"] = df_v1["Close"] - df_v1["Close_365d_ra"]

        df_v1["diff_DRA_7d"] = df_v1["Close"] - df_v1["DA_7d_ra"]
        df_v1["diff_DRA_30d"] = df_v1["Close"] - df_v1["DA_30d_ra"]
        df_v1["diff_DRA_90d"] = df_v1["Close"] - df_v1["DA_90d_ra"]
        df_v1["diff_DRA_365d"] = df_v1["Close"] - df_v1["DA_365d_ra"]
        ''' 

        df_v1.to_csv("Cleaned/" + os.path.basename(f))


        
        '''
        #By year
        

        #Averaging each variable by year
        byYear_df = df["Close"].resample("Y").mean()
        byYear_df = pd.DataFrame(byYear_df)
        byYear_df["Open"] = df["Open"].resample("Y").mean()
        byYear_df["High"] = df["High"].resample("Y").mean()
        byYear_df["Low"] = df["Low"].resample("Y").mean()
        byYear_df["Volume"] = df["Volume"].resample("Y").mean()
        byYear_df["OpenInt"] = df["OpenInt"].resample("Y").mean()

        byYear_df["avg_price"] = (byYear_df["Open"] + byYear_df["High"] + byYear_df["Low"] + byYear_df["Close"])/4

        #Target variable: Percentage change
        byYear_df["percentage_change"] = (byYear_df["Close"].diff()/byYear_df["Close"].shift())*100

        #3 year, 5 year rolling mean, and difference of close from the rolling means
        byYear_df["3y_mean_close"] = byYear_df["Close"].rolling(3).mean()
        byYear_df["5y_mean_close"] = byYear_df["Close"].rolling(5).mean()
        byYear_df["diff_from_3y_close"] = byYear_df["Close"]-byYear_df["3y_mean_close"]
        byYear_df["diff_from_5y_close"] = byYear_df["Close"]-byYear_df["5y_mean_close"]

        byYear_df["3y_mean_avg"] = byYear_df["avg_price"].rolling(3).mean()
        byYear_df["5y_mean_avg"] = byYear_df["avg_price"].rolling(5).mean()
        byYear_df["diff_from_3y_avg"] = byYear_df["Close"]-byYear_df["3y_mean_avg"]
        byYear_df["diff_from_5y_avg"] = byYear_df["Close"]-byYear_df["5y_mean_avg"]

        #copy target variable to use as lag variable of target itself, shift target variable forward
        byYear_df["perct_change_lag"] = byYear_df["percentage_change"]
        byYear_df["percentage_change"] = byYear_df["percentage_change"].shift(periods = -1) #Target variable will be 1 year ahead of all other variables EXCEPT DATE
        
        byYear_df.to_csv("Cleaned/" + os.path.basename(f))
        '''




In [ ]:
def bband(std, mean, price):
    diff = abs(price - mean)
    margin = 2*std
    if price <= mean:
        return -1 * ((diff/margin)*100)
    if price > mean:
        return (diff/margin)*100

In [ ]:
df["20d_ra"] = df["daily_avg"].rolling(window=20).mean()
df["20d_std"] = df["daily_avg"].rolling(window=20).std()
df = df.dropna()
df["dev_from_band"] = [bband(df["20d_std"][i], df["20d_ra"][i], df["Close"][i]) for i in range(len(df))]
df["dev_from_band"] = df["dev_from_band"].fillna(0)

#creating self lag, and target variables if needed
df["close_diff_lag"] = df["Close"].diff()
df["close_diff_target"] = df["close_diff_lag"].shift(periods = -1)
df= df.dropna()